In [1]:
# import filepath
import os

# 设置 DISPLAY
os.environ["DISPLAY"] = ":0"
from direct.showbase.ShowBase import ShowBase
from panda3d.core import WindowProperties



In [2]:
import filepath_p2

In [3]:
from panda3d.core import (
    NodePath,
    WindowProperties,
    Vec3,
    TextNode,
    PNMImage, Texture,
    CardMaker,Point2,
    NodePath, Camera, PerspectiveLens,
    Point3, LVector3f
)

In [4]:
from art.basic import (
    create_cube_node, create_sphere_node,
uv_curve_surface,create_colored_cube_node
)

In [5]:
sphere_vert_diffuse = """
#version 140

in vec4 p3d_Vertex;
in vec3 p3d_Normal;

uniform mat4 p3d_ModelViewMatrix;
uniform mat4 p3d_ModelViewProjectionMatrix;
uniform mat3 p3d_NormalMatrix;

out vec3 fragPos;
out vec3 normal;
out vec3 normRaw;


void main() {
    // Calculate vertex position, fragment position, and surface normal
    gl_Position = p3d_ModelViewProjectionMatrix * p3d_Vertex;
    fragPos = vec3(p3d_ModelViewMatrix * p3d_Vertex);
    normal = p3d_NormalMatrix * p3d_Normal;
    normRaw = p3d_Normal;
}
"""

In [6]:
sphere_frag_0 = """
#version 140

in vec3 fragPos;
in vec3 normRaw;

uniform struct p3d_LightModelParameters {
    vec4 ambient;
} p3d_LightModel;
uniform struct p3d_LightSourceParameters {
    // Primary light color.
    vec4 color;

    // Light color broken up into components, for compatibility with legacy
    // shaders. These are now deprecated.
    vec4 ambient;
    vec4 diffuse;
    vec4 specular;

    // View-space position. If w=0, this is a directional light, with the xyz
    // being -direction.
    vec4 position;

    // Spotlight-only settings
    vec3 spotDirection;
    float spotExponent;
    float spotCutoff;
    float spotCosCutoff;

    // Individual attenuation constants
    float constantAttenuation;
    float linearAttenuation;
    float quadraticAttenuation;

    // constant, linear, quadratic attenuation in one vector
    vec3 attenuation;
    // Shadow map for this light source
    sampler2DShadow shadowMap;

    // Transforms view-space coordinates to shadow map coordinates
    mat4 shadowViewMatrix;
} p3d_LightSource[3];
uniform struct p3d_MaterialParameters {
    vec4 ambient;
    vec4 diffuse;
    vec4 emission;
    vec3 specular;
    float shininess;
    
    vec4 baseColor;
    float roughness;
    float metallic;
    float refractiveIndex;
} p3d_Material;

out vec4 p3d_FragColor;
vec4 normal2color(){
    vec3 norm = normalize(normRaw);
    return vec4(norm * 0.5 + 0.5, 1.0);
    }


void main() {
    // Calculate base color
    vec4 baseColor = vec4(0.5, .5, .8, 1.0);
    // Calculate final color
    p3d_FragColor = normal2color();
}
"""

In [7]:
from util.geometry import format_normal

In [8]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import (
    AmbientLight,
    DirectionalLight,
    PointLight,
    Shader,
    Material,
    Vec3,
    Vec4
)
from panda3d_game.app import ContextShowBase
# Shader
# Application Class
# =================
normals = []

class ShaderDemo(ContextShowBase):
    def __init__(self):
        # Call the base constructor
        ContextShowBase.__init__(self)
        self.sphere = create_sphere_node("sphere", lat_res=24, lon_res=24,vformat=format_normal)
        self.sphere_np = NodePath(self.sphere)
        self.indicator = create_sphere_node("sphere", lat_res=4, lon_res=24)
        self.indicator_np = NodePath(self.indicator)
        self.indicator_np.set_scale(.1)
        self.indicator_np.reparent_to(self.render)


        # 创建材质

        mat = Material()
        mat.setAmbient((1, 1, 1, 1))# 环境光反射
        mat.setSpecular((0.5, 0.5, 0.5, 1))# 镜面光颜色
        mat.setShininess(20)# 镜面高光大小/锐利程度
        mat.setDiffuse((0.8, 0.8, 0.8, 1))  # RGB 越接近 1 越亮/浅  # 漫反射
        
        
        # self.sphere_np.setMaterial(mat)
        

        self.ambient_light = self.render.attach_new_node(AmbientLight("AmbientLight"))
        # self.ambient_light.node().set_color(Vec4(.2, .2, .2, 1))
        self.ambient_light.node().set_color(Vec4(.2, .2, .2, 1))
        self.render.set_light(self.ambient_light)
        
        self.sun = self.render.attach_new_node(DirectionalLight("Sun"))
        self.sun.set_hpr(45, -45, 0)
        self.render.set_light(self.sun)
        
        self.green_light = self.render.attach_new_node(PointLight("GreenLight"))
        self.green_light.node().set_color(Vec4(0, 1, 0, 1))
        self.green_light.node().set_attenuation(Vec3(1, .1, .5))
        # self.green_light.node().set_attenuation(Vec3(1, 0.1, 0.05))
        # self.green_light.node().set_attenuation(Vec3(1,0,0))
        self.green_light.set_pos(0, 3, 0)
        self.indicator_np.set_pos(0,3,0)
        self.render.set_light(self.green_light)

        geom = self.sphere_np.node().getGeom(0)
        vdata = geom.getVertexData()
        print(vdata.hasColumn("normal"))
        from panda3d.core import GeomVertexReader
        normal_reader = GeomVertexReader(vdata, "normal")
        while not normal_reader.isAtEnd():
            n = normal_reader.getData3f()
            normals.append(n)
        print(normals[:10])

        self.sphere_shader = Shader.make(
            Shader.SL_GLSL,
            sphere_vert_diffuse,
            # sphere_frag_diffuse
            sphere_frag_0
            # specular_vert,
            # specular_frag
            # sphere_vert,
            # sphere_frag_ambient
        )
        self.sphere_np.set_pos(0, 5, 0)
        self.sphere_np.set_shader(self.sphere_shader)
        self.sphere_np.reparent_to(self.render)

    def pause_switch(self, *args,**kwargs):
        pass





In [9]:
from qpanda3d import QShowBase, QPanda3DWidget, QControl, Synchronizer
from demos.physics_room import PhyscRoomConsole
class ShaderControl(ShaderDemo, QControl):
    def __init__(self, isQt = True):
        import pdb
        QControl.__init__(self)
        ShaderDemo.__init__(self)
        self.isQt = isQt
        if self.isQt:
            self.startQt()
from demos.physics_room import PhyscRoomConsole
from ui.qtui import *

class ShaderGame(RawQtGUI):
    def get_game(self):
        return ShaderControl()

    def get_console(self):
        return PhyscRoomConsole(showbase=self.panda3d)

if __name__ == '__main__':
    # torch.set_printoptions(precision=16, sci_mode=False)
    import sys
    app = QApplication(sys.argv)
    window = ShaderGame()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4381.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


True
[LVecBase3f(1, 0, 0), LVecBase3f(0.965926, 0, 0.258819), LVecBase3f(0.866025, 0, 0.5), LVecBase3f(0.707107, 0, 0.707107), LVecBase3f(0.5, 0, 0.866025), LVecBase3f(0.258819, 0, 0.965926), LVecBase3f(0, 0, 1), LVecBase3f(-0.258819, 0, 0.965926), LVecBase3f(-0.5, 0, 0.866025), LVecBase3f(-0.707107, 0, 0.707107)]


SystemExit: 0

/mnt/D/packages/miniconda3/envs/game/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
